In [1]:
import tensorflow as tf
#import tensorflow.nn.rnn_cell as rnn
import numpy as np
#import tensorflow.keras as keras
#from tensorflow.keras.initializers import Constant
import tensorflow.keras.backend as K


from tqdm import tqdm
import os

In [2]:
#load 100hz data
data_x=np.loadtxt('./PSDSData (24)_raw.csv',delimiter=",",dtype=np.float32)
data_y=np.loadtxt('./Tap(24)_y_100Hz.csv',delimiter=",",dtype=np.float32)
preY = np.loadtxt('./Tap(24)_y_100Hz.csv',delimiter=",",dtype=np.float32)
#data_x=data_x[:,3:9];
#data_y=np.zeros(len(data_x))
#preY=data_y
data_x.shape,data_y.shape

((120088, 9), (120088,))

In [3]:
#reduce from 100hz to 25hz
new_x=np.zeros_like(data_x[0:1])
new_y=np.zeros_like(data_y[0:1])
for i in range(int(len(data_x)/4)):
    temp=data_x[ i*4 : (i+1)*4 ].mean(axis=0)
    for j in range(len(temp)):
        if temp[j] >= 0:
            temp[j]=data_x[ i*4 : (i+1)*4 ].max(axis=0)[j]
        else:
            temp[j]=data_x[ i*4 : (i+1)*4 ].min(axis=0)[j]
    new_x=np.append(new_x,[temp],axis=0)
    new_y=np.append(new_y,data_y[i*4:(i+1)*4].max())
new_x=np.delete(new_x,0,axis=0)
new_y=np.delete(new_y,0,axis=0)

print(new_x.shape,new_y.shape)

input_data=new_x[:,3:6]
x_states=np.zeros([1,128],dtype=np.float32)
output_data=np.zeros(len(input_data))

input_data.shape,x_states.shape,output_data.shape

(30022, 9) (30022,)


((30022, 3), (1, 128), (30022,))

In [4]:
interpreter=tf.lite.Interpreter(model_path="tapDetectorLSTM.tflite")
#interpreter=tf.lite.Interpreter(model_path="tap_9_24.tflite")
interpreter.allocate_tensors()
input_details=interpreter.get_input_details()
output_details=interpreter.get_output_details()

In [5]:
TimeDiffThreshold=3  # TapLockoutTimeMs=100ms

jerkThresholdDynamic=30.0 # no use
jerkThresholdOnOffDiff=10.0 # no use

jerkThreshold=30.0  # 17.0
maxJerkThreshold=70.0
minJerkThreshold=30.0
systemVersionJerkFactor=3.5

predictionThreshold=0.5
minPredictionThreshold=0.5
maxPredictionThreshold=1.0
predictionThresholdOnOffDiff=0.2 # no use
predictionThresholdDynamic=0.5 # no use

InputHistorySize=4
inputHistory=[None]*InputHistorySize

PredictionHistorySize=2
predictionHistory=[None]*PredictionHistorySize

output_tap=[None]*len(input_data)

InputRawHistorySize=6
inputRawHistory=[None]*InputRawHistorySize

jerk=[None]*len(input_data)
JerkHistorySize=2
jerkHistory=np.zeros(JerkHistorySize)

In [6]:
def updateInputHistory(input1):
    for i in range(InputHistorySize-1,0,-1):
        inputHistory[i]=inputHistory[i-1]
    inputHistory[0]=input1
        
def updatePredictions(pred):
    for i in range(PredictionHistorySize-1,0,-1):
        predictionHistory[i]=predictionHistory[i-1]
    predictionHistory[0]=pred

def updateInputRawHistory(input1):
    for i in range(InputRawHistorySize-1,0,-1):
        inputRawHistory[i]=inputRawHistory[i-1]
    inputRawHistory[0]=input1
    
def updateJerkHistory(input1):
    for i in range(JerkHistorySize-1,0,-1):
        jerkHistory[i]=jerkHistory[i-1]
    jerkHistory[0]=input1

def didTap(i,lastTap,tapGap):
    if lastTap != -1:
        timeDiff=i-lastTap
        if timeDiff < TimeDiffThreshold:
            tap=0
            return tap,lastTap,0
    
    if (inputHistory[InputHistorySize-1] is None) or (predictionHistory[PredictionHistorySize-1] is None):
        tap=0
        return tap,lastTap,0
    
    #raw data max-min > jerkThreshold
    minZ=np.array(inputRawHistory).min(axis=0)[2]
    maxZ=np.array(inputRawHistory).max(axis=0)[2]
    jerk=maxZ-minZ
    
    updateJerkHistory(jerk)
    
    maxJerk=np.array(jerkHistory).max()

    if (maxJerk>jerkThreshold) & all(pred > predictionThreshold for pred in predictionHistory) & tapGap:
        tap=1
        lastTap=i
    elif(maxJerk>maxJerkThreshold)&all(pred>minPredictionThreshold for pred in predictionHistory)&tapGap:
        tap=1
        lastTap=i
    else:
        tap=0
    
    #print(jerk,predictionHistory[0])
    return tap,lastTap,jerk

def calJerkThresholdDynamic():
    _maxJerk=maxJerkThreshold;
    _minJerk=minJerkThreshold;
    _maxPrediction=maxPredictionThreshold;
    _minPrediction=minPredictionThreshold;
    
    # old system
    if(systemUpToDate==False):
        _maxJerk = maxJerkThreshold / systemVersionJerkFactor
        _minJerk = minJerkThreshold / systemVersionJerkFactor
    
    jerkThreshold = _maxJerk - (_maxJerk - _minJerk) * scaleFactor
    predictionThrehold = _maxPrediction - (_maxPrediction - _minPrediction) * scaleFactor
    print(jerkThreshold,predictionThrehold)
    

def verify(taps,label_taps):
    False_positive= np.zeros_like(taps)
    False_negtive = np.zeros_like(taps)
    True_positive = np.zeros_like(taps)
    window=5
    #taps=np.append(np.zeros([1]),taps)
    #taps=np.append(taps,np.zeros([1]))
    #label_taps=np.append(np.zeros([1]),label_taps)
    #label_taps=np.append(label_taps,np.zeros([1]))
    for i in range(len(taps)):
        if label_taps[i] ==1:
            if taps[i-window:i+window].sum()==0:
                False_negtive[i]=1
            else:
                True_positive[i]=1

        if taps[i] ==1:
            if label_taps[i-window:i+window].sum()==0:
                False_positive[i]=1
                #raw_buff=raw[i-4:i+1,2]# only z acc
                #diff=abs(raw_buff.max()-raw_buff.min())
                #print(i,raw_buff,diff)
            
    return [False_positive,  False_negtive ,    True_positive ]

In [7]:
lastTap=-1
tapGap=True
systemUpToDate=True
scaleFactor = 0.8
calJerkThresholdDynamic()

for i in tqdm(range(len(input_data))):
    # input_data: 25hz
    input_x=input_data[i,:].reshape([1,3])
    updateInputHistory(input_x)
    for j in range(4):
        # data_x: 100hz
        updateInputRawHistory(data_x[i*4+j,3:6])
    
    interpreter.set_tensor(input_details[0]['index'],x_states)
    interpreter.set_tensor(input_details[1]['index'],input_x)
    interpreter.invoke()
    
    output_data[i]=interpreter.get_tensor(output_details[1]['index'])
    x_states=interpreter.get_tensor(output_details[0]['index'])
    
    updatePredictions(output_data[i])    
     
    output_tap[i],lastTap,jerk[i]=didTap(i,lastTap,tapGap)
    
    if (lastTap > -1) & all(pred < predictionThreshold for pred in predictionHistory):
        tapGap=True
    elif lastTap==i:
        tapGap=False

  8%|▊         | 2347/30022 [00:00<00:01, 23463.87it/s]

38.0 0.6


100%|██████████| 30022/30022 [00:01<00:00, 25177.39it/s]


In [8]:
False_positive,  False_negtive ,    True_positive=verify(np.array(output_tap),new_y)

False_positive.sum(),  False_negtive.sum() ,    True_positive.sum()

(0, 88, 847)

In [13]:
np.savetxt('output_tap.csv',output_tap)

np.savetxt('jerk.csv',jerk)

np.savetxt('fp.csv',False_positive)

np.savetxt('fn.csv',False_negtive)

np.savetxt('tp.csv',True_positive)

np.savetxt('output_data.csv',output_data)

In [10]:
sum(1 for i in jerk if i >60)

41

In [9]:
sum(1 for i in jerk if i >30)

484

In [11]:
sum(1 for i in jerk if i >10)

671